In [107]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 200)
pd.options.display.float_format = '{:,.0f}'.format
from __future__ import division

In [14]:
columns = ['id','scrape_id', 'neighbourhood', 'zipcode', 'property_type', 'room_type', 'accommodates', 'bedrooms', 'beds', 'price', 'number_of_reviews', 'review_scores_rating', 'reviews_per_month'] 
listings = pd.read_csv('listings.csv')
#listings1 = listings(copy=True)
listings = listings[columns]
listings.index = listings['id']
del listings['id']
listings['price'] = listings['price'].replace(['[\$]', '[\,]'], '', regex=True).apply(pd.to_numeric)
# --> filter zipcode on first 4 numbers
listings['zipcode'] = listings['zipcode'].str[:4].apply(pd.to_numeric, errors='coerce')
listings.dropna(subset=['zipcode'], inplace=True)
# convert dat zipcode to a prupper int
listings['zipcode'] = listings['zipcode'].apply(lambda x: int(x))


listings['zipcode'].iloc[0]

1054

In [3]:
#price & reviews 
listings['zipcode'].astype('category')
price_reviews = pd.pivot_table(listings, index=['zipcode'], values=['price', 'scrape_id','number_of_reviews', 'review_scores_rating', 'reviews_per_month'], aggfunc={'price': np.mean, 'scrape_id': len, 'review_scores_rating': np.mean, 'reviews_per_month': np.mean, 'number_of_reviews': np.sum})
del price_reviews['scrape_id']
# --->>> JSON
price_reviews.sort() # SORT
price_reviews.to_json('price_reviews.json', orient='index')

/Users/sjoerd/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(....) is deprecated, use sort_index(.....)


In [155]:
# PROPERTY TYPES PIVOT

property_types = pd.pivot_table(listings, index='zipcode', columns='property_type', values='scrape_id', aggfunc=len).fillna(value=0)
property_types = pd.DataFrame(property_types)

property_types['total'] = property_types.sum(axis=1)
#property_types = property_types.loc[:, 'Apartment':'Villa'].div(property_types.total, axis = 0)
# --->>> JSON
p_columns = ['Apartment', 'Bed & Breakfast', 'Boat', 'Condominium', 'House', 'Loft', 'Townhouse']
property_types[p_columns].to_json('property_types.json', orient='index')

In [156]:
price_reviews

,number_of_reviews,price,review_scores_rating,reviews_per_month
zipcode,,,,
0,1,120,97,0
173,21,150,96,2
555,12,110,98,1
689,3,310,100,0
1000,0,74,nan,nan
1010,34,125,95,1
1011,7238,177,94,2
1012,13247,180,92,2
1013,11779,142,94,1


In [154]:
listings

,scrape_id,neighbourhood,zipcode,property_type,room_type,accommodates,bedrooms,beds,price,number_of_reviews,review_scores_rating,reviews_per_month
id,,,,,,,,,,,,
14831696,20170402075052,NaN,1054,Apartment,Entire home/apt,6,2,6,145,4,95,1
3951251,20170402075052,NaN,1057,Apartment,Entire home/apt,2,1,1,88,17,93,1
16590973,20170402075052,NaN,1054,Apartment,Entire home/apt,2,1,1,130,4,100,2
9283701,20170402075052,Oud-West,1057,Apartment,Entire home/apt,4,2,3,110,2,100,0
6458721,20170402075052,Oud-West,1053,Apartment,Entire home/apt,2,1,1,115,10,96,0
14864447,20170402075052,Oud-West,1054,Apartment,Private room,1,1,1,70,11,83,2
16306240,20170402075052,Oud-West,1053,House,Entire home/apt,2,1,1,120,3,100,1
15211538,20170402075052,NaN,1056,Apartment,Private room,2,1,1,45,33,98,5
1261425,20170402075052,Oud-West,1056,Bed & Breakfast,Private room,2,1,nan,45,2,97,0
